In [1]:
import os
#import jovian
import matplotlib
import opendatasets as od
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
%matplotlib inline
sns.set_style('darkgrid')
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (10, 6)
matplotlib.rcParams['figure.facecolor'] = '#00000000'

In [2]:
merged_df = pd.read_csv('merged_data_2.csv')

In [3]:
submission_df = pd.read_csv('sample submission 3.csv')

In [4]:
submission_df.nunique()

MAGIC_KEY    5379
MEAT            1
dtype: int64

In [5]:
merged_df['PURCHASE_DATE'] = pd.to_datetime(merged_df['PURCHASE_DATE'])
#merged_df['LAST_PURCHASE_DATE'] = pd.to_datetime(merged_df['LAST_PURCHASE_DATE'])

### Encoding Categorical Data

In [6]:
from sklearn.preprocessing import LabelEncoder
import pickle

In [7]:
le = LabelEncoder() #creating an instance of LabelEncoder
merged_df['MAGIC_KEY_ENCODED'] = le.fit_transform(merged_df['MAGIC_KEY'])
merged_df['DELIVERY_ENCODED'] = le.fit_transform(merged_df['DELIVERY_OPTION'])
merged_df['QUALITY_ENCODED'] = le.fit_transform(merged_df['QUALITY'])

In [8]:
prb_df_without_encode = pd.read_csv('problem 3.csv')
prb_df_encode = prb_df_without_encode.copy()

In [9]:
prb_df_encode = prb_df_encode.merge(merged_df, on='MAGIC_KEY', how='left').drop_duplicates(subset=['MAGIC_KEY'])


In [10]:
#prb_df_encode.MAGIC_KEY.value_counts()

In [11]:
#prb_df_encode

## Added magic key encode in the submission df

In [12]:
prb_df_encode['MAGIC_KEY_ENCODED'] = le.fit_transform(prb_df_encode['MAGIC_KEY'])


In [13]:
### drop purchase date and last purchase date column
#prb_df_encode = prb_df_encode.drop(columns=['MAGIC_KEY'])

In [14]:
prb_df_encode.duplicated().sum()

0

In [15]:
#prb_df_encode.info()

In [16]:
merged_df['MAGIC_KEY_ENCODED'] = le.fit_transform(merged_df['MAGIC_KEY'])


## splitting dataset for training and val set

In [17]:
len(merged_df)

2455770

In [18]:
train_size = int(.75 * len(merged_df))
train_size

1841827

In [19]:
sorted_df = merged_df.sort_values('PURCHASE_DATE')
train_df, val_df = sorted_df[:train_size], sorted_df[train_size:]

In [20]:
len(train_df), len(val_df)

(1841827, 613943)

In [21]:
val_df.PURCHASE_DATE.min(), val_df.PURCHASE_DATE.max()

(Timestamp('2019-01-30 00:00:00'), Timestamp('2019-02-28 00:00:00'))

In [22]:
train_df.head()

,BOX_ID,QUALITY,DELIVERY_OPTION,MILK,MEAT,UNIT_PRICE,PURCHASE_DATE,MAGIC_KEY,BOX_COUNT,TOTAL_MEAT,...,02_LAST_PURCHASE_2019,PURCHASE_DURATION,AVG_MEAT_PURCHASE,PURCHASE_DAY_OF_WEEK,PREVIOUS_MEAT_QUANTITY,MEAT_MEAN,MEAT_MEDIAN,MAGIC_KEY_ENCODED,DELIVERY_ENCODED,QUALITY_ENCODED
2333195,260,Standard,Delivery from Collection Point,10.0,1.8,17.98,2018-10-01,29339036B48,1.0,4.0,...,0.0,25.0,2.000,0,2.2,2.000,2.00,236134,0,1
736244,118,Premium,Delivery from Collection Point,8.0,1.3,15.98,2018-10-01,2CA943107B8,1.0,3.8,...,0.0,81.0,1.900,0,2.5,1.900,1.90,1004944,0,0
736245,118,Premium,Delivery from Collection Point,8.0,1.3,15.98,2018-10-01,2C00FF1E2DC,1.0,3.1,...,0.0,0.0,1.550,0,2.5,1.550,1.55,562284,0,0
736246,118,Premium,Delivery from Collection Point,8.0,1.3,15.98,2018-10-01,2C1A276B78F,1.0,8.5,...,0.0,0.0,1.700,0,2.5,1.700,1.80,636285,0,0
736247,118,Premium,Delivery from Collection Point,8.0,1.3,15.98,2018-10-01,2BD8A103BA3,1.0,5.7,...,0.0,114.0,1.425,0,1.3,1.425,1.30,421096,0,0


# train, test, val split

In [23]:
#train_df.info()

In [24]:
### drop purchase date and last purchase date column
train_df = train_df.drop(columns=['PURCHASE_DATE', 'MAGIC_KEY', 'QUALITY', 'DELIVERY_OPTION'])
val_df = val_df.drop(columns=['PURCHASE_DATE', 'MAGIC_KEY', 'QUALITY'])
test_df = prb_df_encode.drop(columns=['PURCHASE_DATE', 'MAGIC_KEY', 'QUALITY'])

In [25]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1841827 entries, 2333195 to 815824
Data columns (total 34 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   BOX_ID                  int64  
 1   MILK                    float64
 2   MEAT                    float64
 3   UNIT_PRICE              float64
 4   BOX_COUNT               float64
 5   TOTAL_MEAT              float64
 6   PURCHASE_FREEQUENCY     int64  
 7   DAY                     int64  
 8   MONTH                   int64  
 9   YEAR                    int64  
 10  OCT_MEAT                float64
 11  NOV_MEAT                float64
 12  DEC_MEAT                float64
 13  JAN_MEAT                float64
 14  FEB_MEAT                float64
 15  10_FIRST_PURCHASE_2018  float64
 16  10_LAST_PURCHASE_2018   float64
 17  11_FIRST_PURCHASE_2018  float64
 18  11_LAST_PURCHASE_2018   float64
 19  12_FIRST_PURCHASE_2018  float64
 20  12_LAST_PURCHASE_2018   float64
 21  01_FIRST_PURCHASE_2019  float64

In [26]:
train_df.shape, val_df.shape, test_df.shape

((1841827, 34), (613943, 35), (5379, 35))

In [27]:
train_df.columns

Index(['BOX_ID', 'MILK', 'MEAT', 'UNIT_PRICE', 'BOX_COUNT', 'TOTAL_MEAT',
       'PURCHASE_FREEQUENCY', 'DAY', 'MONTH', 'YEAR', 'OCT_MEAT', 'NOV_MEAT',
       'DEC_MEAT', 'JAN_MEAT', 'FEB_MEAT', '10_FIRST_PURCHASE_2018',
       '10_LAST_PURCHASE_2018', '11_FIRST_PURCHASE_2018',
       '11_LAST_PURCHASE_2018', '12_FIRST_PURCHASE_2018',
       '12_LAST_PURCHASE_2018', '01_FIRST_PURCHASE_2019',
       '01_LAST_PURCHASE_2019', '02_FIRST_PURCHASE_2019',
       '02_LAST_PURCHASE_2019', 'PURCHASE_DURATION', 'AVG_MEAT_PURCHASE',
       'PURCHASE_DAY_OF_WEEK', 'PREVIOUS_MEAT_QUANTITY', 'MEAT_MEAN',
       'MEAT_MEDIAN', 'MAGIC_KEY_ENCODED', 'DELIVERY_ENCODED',
       'QUALITY_ENCODED'],
      dtype='object')

### declaring input and target cols for train, val, test set

In [28]:
input_cols = [col for col in train_df.columns if col != 'FEB_MEAT']

print(input_cols)


['BOX_ID', 'MILK', 'MEAT', 'UNIT_PRICE', 'BOX_COUNT', 'TOTAL_MEAT', 'PURCHASE_FREEQUENCY', 'DAY', 'MONTH', 'YEAR', 'OCT_MEAT', 'NOV_MEAT', 'DEC_MEAT', 'JAN_MEAT', '10_FIRST_PURCHASE_2018', '10_LAST_PURCHASE_2018', '11_FIRST_PURCHASE_2018', '11_LAST_PURCHASE_2018', '12_FIRST_PURCHASE_2018', '12_LAST_PURCHASE_2018', '01_FIRST_PURCHASE_2019', '01_LAST_PURCHASE_2019', '02_FIRST_PURCHASE_2019', '02_LAST_PURCHASE_2019', 'PURCHASE_DURATION', 'AVG_MEAT_PURCHASE', 'PURCHASE_DAY_OF_WEEK', 'PREVIOUS_MEAT_QUANTITY', 'MEAT_MEAN', 'MEAT_MEDIAN', 'MAGIC_KEY_ENCODED', 'DELIVERY_ENCODED', 'QUALITY_ENCODED']


In [29]:

target_col = 'FEB_MEAT'

In [30]:
train_inputs = train_df[input_cols].copy()
train_targets = train_df[target_col].copy()

In [31]:
val_inputs = val_df[input_cols].copy()
val_targets = val_df[target_col].copy()

In [32]:
test_inputs = test_df[input_cols].copy()
# Test data does not have targets

### Imputing missing numeric values

In [33]:
from sklearn.impute import SimpleImputer

In [34]:
imputer = SimpleImputer(strategy = 'mean').fit(merged_df[input_cols])

In [35]:
train_inputs[input_cols] = imputer.transform(train_inputs[input_cols])
val_inputs[input_cols] = imputer.transform(val_inputs[input_cols])
test_inputs[input_cols] = imputer.transform(test_inputs[input_cols])

### checking the null values

In [36]:
test_inputs[input_cols].isna().sum()

BOX_ID                    0
MILK                      0
MEAT                      0
UNIT_PRICE                0
BOX_COUNT                 0
TOTAL_MEAT                0
PURCHASE_FREEQUENCY       0
DAY                       0
MONTH                     0
YEAR                      0
OCT_MEAT                  0
NOV_MEAT                  0
DEC_MEAT                  0
JAN_MEAT                  0
10_FIRST_PURCHASE_2018    0
10_LAST_PURCHASE_2018     0
11_FIRST_PURCHASE_2018    0
11_LAST_PURCHASE_2018     0
12_FIRST_PURCHASE_2018    0
12_LAST_PURCHASE_2018     0
01_FIRST_PURCHASE_2019    0
01_LAST_PURCHASE_2019     0
02_FIRST_PURCHASE_2019    0
02_LAST_PURCHASE_2019     0
PURCHASE_DURATION         0
AVG_MEAT_PURCHASE         0
PURCHASE_DAY_OF_WEEK      0
PREVIOUS_MEAT_QUANTITY    0
MEAT_MEAN                 0
MEAT_MEDIAN               0
MAGIC_KEY_ENCODED         0
DELIVERY_ENCODED          0
QUALITY_ENCODED           0
dtype: int64

# Scaling Numeric Features

In [37]:
#from sklearn.preprocessing import MinMaxScaler

In [38]:
#scaler = MinMaxScaler().fit(merged_df[input_cols])

In [39]:
# train_inputs[input_cols] = scaler.transform(train_inputs[input_cols])
# val_inputs[input_cols] = scaler.transform(val_inputs[input_cols])
# test_inputs[input_cols] = scaler.transform(test_inputs[input_cols])

In [40]:
#val_inputs.describe().loc[['min', 'max']]

In [41]:
#test_inputs.head()

## X_train, X_val, X_test

In [42]:
X_train = train_inputs
X_val = val_inputs
X_test = test_inputs

In [43]:
X_train.head()

,BOX_ID,MILK,MEAT,UNIT_PRICE,BOX_COUNT,TOTAL_MEAT,PURCHASE_FREEQUENCY,DAY,MONTH,YEAR,...,02_LAST_PURCHASE_2019,PURCHASE_DURATION,AVG_MEAT_PURCHASE,PURCHASE_DAY_OF_WEEK,PREVIOUS_MEAT_QUANTITY,MEAT_MEAN,MEAT_MEDIAN,MAGIC_KEY_ENCODED,DELIVERY_ENCODED,QUALITY_ENCODED
2333195,260.0,10.0,1.8,17.98,1.0,4.0,2.0,1.0,10.0,2018.0,...,0.0,25.0,2.000,0.0,2.2,2.000,2.00,236134.0,0.0,1.0
736244,118.0,8.0,1.3,15.98,1.0,3.8,2.0,1.0,10.0,2018.0,...,0.0,81.0,1.900,0.0,2.5,1.900,1.90,1004944.0,0.0,0.0
736245,118.0,8.0,1.3,15.98,1.0,3.1,2.0,1.0,10.0,2018.0,...,0.0,0.0,1.550,0.0,2.5,1.550,1.55,562284.0,0.0,0.0
736246,118.0,8.0,1.3,15.98,1.0,8.5,5.0,1.0,10.0,2018.0,...,0.0,0.0,1.700,0.0,2.5,1.700,1.80,636285.0,0.0,0.0
736247,118.0,8.0,1.3,15.98,1.0,5.7,4.0,1.0,10.0,2018.0,...,0.0,114.0,1.425,0.0,1.3,1.425,1.30,421096.0,0.0,0.0


## model training

In [44]:
#from sklearn.tree import DecisionTreeRegressor


In [45]:
# Assuming model is DecisionTreeRegressor
#model = DecisionTreeRegressor(random_state=42)

In [46]:
# %%time
# # Fit the model
# model.fit(X_val, val_targets)

### Evaluation

Let's evaluate the decision tree using the accuracy score.

In [47]:
from sklearn.metrics import mean_squared_error
from math import sqrt

# Calculate RMSE
#rmse = sqrt(mean_squared_error(true_values, predicted_values))


In [48]:
# val_preds = model.predict(X_val)
# val_preds

In [49]:
#train_preds

In [50]:
# Calculate RMSE
#rmse = sqrt(mean_squared_error(train_preds, train_targets )) 

In [51]:
#round(rmse)

### Baseline ML model

Let's train a simple `LinearRegression` model, with no customization.

In [52]:
from sklearn.linear_model import LinearRegression

In [53]:
linreg = LinearRegression()

In [54]:
linreg.fit(X_train, train_targets)

LinearRegression()

In [55]:
train_preds = linreg.predict(X_train)
train_preds

array([-1.82613779e-12,  2.50000000e+00, -2.36954127e-13, ...,
       -2.10244683e-12,  4.20000000e+00, -3.37908888e-13])

In [56]:
mean_squared_error(train_preds, train_targets, squared=False)

3.0421678106425786e-12

In [57]:
linreg.fit(X_val, val_targets)

LinearRegression()

In [58]:
val_preds = linreg.predict(X_val)
val_preds

array([-5.51591199e-13,  1.80000000e+00,  1.80000000e+00, ...,
        2.52548261e-13,  8.07389648e-13,  1.47518194e-13])

In [59]:
mean_squared_error(val_preds, val_targets, squared=False)

5.017904499423005e-13

## Step 6 - Pick a strategy, train a model & tune hyperparameters

<img src="https://i.imgur.com/aRuE5mw.png" width="640">


### Systematically Exploring Modeling Strategies

Scikit-learn offers the following cheatsheet to decide which model to pick.

![](https://scikit-learn.org/stable/_static/ml_map.png)


Here's the general strategy to follow:

- Find out which models are applicable to the problem you're solving.
- Train a basic version for each type of model that's applicable
- Identify the modeling approaches that work well and tune their hypeparameters
- [Use a spreadsheet](https://docs.google.com/spreadsheets/d/1S42lnklqdj4hS3BohffyvZVJOFBZJ61VDiHLMuf2zBw/edit?usp=sharing) to keep track of your experiments and results.

Let's define a function `try_model`, which takes a model, then performs training and evaluation.

In [60]:
def try_model(model):
    # Fit the model
    model.fit(X_train, train_targets)
    
    # Generate predictions
    train_preds = model.predict(X_train)
    val_preds = model.predict(X_val)
    #test_preds = model.predict(X_test)
    
    # Compute RMSE
    train_rmse = sqrt(mean_squared_error(train_targets, train_preds, squared=False))
    val_rmse = sqrt(mean_squared_error(val_targets, val_preds, squared=False))
    #test_rmse = sqrt(mean_squared_error(test_targets, test_preds, squared=False))
    return train_rmse, val_rmse #, test_rmse

### Linear Models

Read about linear models here: https://scikit-learn.org/stable/modules/linear_model.html

In [61]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor

In [62]:
try_model(LinearRegression())

(1.7441811289664209e-06, 1.7144658572770988e-06)

In [63]:
try_model(Ridge())

C:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=4.07094e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


(0.0016549885541950644, 0.0018440203178470144)

In [64]:
try_model(Lasso())

(1.0499326743226387, 0.9851628658903655)

In [65]:
try_model(ElasticNet())

(1.0315741461318344, 0.9583879093595981)

In [66]:
try_model(SGDRegressor())

C:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


(2073046491.1189682, 2111886758.6066625)

### Tree Based Models

* Decision trees: https://scikit-learn.org/stable/modules/tree.html
* Random forests and gradient boosting: https://scikit-learn.org/stable/modules/ensemble.html

In [67]:
from sklearn.tree import DecisionTreeRegressor, plot_tree

In [68]:
tree = DecisionTreeRegressor(random_state=42)
try_model(tree)

(3.3088516627623164e-07, 0.715812266095497)

In [69]:
# plt.figure(figsize=(40, 20))
# plot_tree(tree, max_depth=10, filled=True, feature_names=input_cols);

In [70]:
from sklearn.ensemble import RandomForestRegressor

In [71]:
 %%time
rf = RandomForestRegressor(random_state=42, n_jobs=-1, n_estimators= 500,    max_features=45) # removed this : max_leaf_nodes = 2**20,
 try_model(rf)

CPU times: total: 4h 1min 3s
Wall time: 23min 6s


(0.1828920953315025, 0.707713176792743)

In [72]:
rf.feature_importances_

array([9.09873307e-04, 2.92395172e-03, 9.47325279e-02, 3.06598602e-04,
       5.36123617e-06, 4.56056972e-01, 1.57908682e-02, 2.39612561e-03,
       1.53015179e-01, 4.76729731e-06, 7.79005984e-02, 5.58988184e-02,
       5.36678348e-02, 6.64710092e-02, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       2.25264935e-03, 4.70780424e-03, 2.02564311e-04, 7.49200568e-04,
       4.69881260e-03, 4.55223019e-03, 2.50769553e-03, 1.12414704e-04,
       1.36142873e-04])

In [73]:
importance_df = pd.DataFrame({ 
    'feature': input_cols, 
    'importance': rf.feature_importances_
}).sort_values('importance', ascending=False)
importance_df.head(10)

,feature,importance
5,TOTAL_MEAT,0.456057
8,MONTH,0.153015
2,MEAT,0.094733
10,OCT_MEAT,0.077901
13,JAN_MEAT,0.066471
11,NOV_MEAT,0.055899
12,DEC_MEAT,0.053668
6,PURCHASE_FREEQUENCY,0.015791
25,AVG_MEAT_PURCHASE,0.004708
28,MEAT_MEAN,0.004699


### Making a submission on Kaggle

If you're participating in a Kaggle competition, you can generate a submission CSV file and make a submission to check your score on the test set.

In [74]:
test_preds = rf.predict(X_test)
test_preds

array([1.4052, 2.2   , 2.2   , ..., 0.    , 0.    , 0.    ])

In [75]:
test_df['MEAT_PURCHASE'] = test_preds

In [76]:
test_df.MEAT_PURCHASE.min(), test_df.MEAT_PURCHASE.max()

(0.0, 20.950399999999835)

In [81]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5379 entries, 0 to 15326
Data columns (total 36 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   BOX_ID                  5379 non-null   int64  
 1   DELIVERY_OPTION         5379 non-null   object 
 2   MILK                    5379 non-null   float64
 3   MEAT                    5379 non-null   float64
 4   UNIT_PRICE              5379 non-null   float64
 5   BOX_COUNT               5379 non-null   float64
 6   TOTAL_MEAT              5379 non-null   float64
 7   PURCHASE_FREEQUENCY     5379 non-null   int64  
 8   DAY                     5379 non-null   int64  
 9   MONTH                   5379 non-null   int64  
 10  YEAR                    5379 non-null   int64  
 11  OCT_MEAT                5379 non-null   float64
 12  NOV_MEAT                5379 non-null   float64
 13  DEC_MEAT                5379 non-null   float64
 14  JAN_MEAT                5379 non-null   floa

In [85]:
test_df.head()

,BOX_ID,DELIVERY_OPTION,MILK,MEAT,UNIT_PRICE,BOX_COUNT,TOTAL_MEAT,PURCHASE_FREEQUENCY,DAY,MONTH,...,PURCHASE_DURATION,AVG_MEAT_PURCHASE,PURCHASE_DAY_OF_WEEK,PREVIOUS_MEAT_QUANTITY,MEAT_MEAN,MEAT_MEDIAN,MAGIC_KEY_ENCODED,DELIVERY_ENCODED,QUALITY_ENCODED,MEAT_PURCHASE
0,56,Delivery from Collection Point,10.0,3.6,15.00,1.0,18.0,5,23,2,...,0.0,3.60,5,2.2,3.60,3.6,1334,0,0,1.4052
5,132,Delivery from Collection Point,9.0,1.8,19.98,1.0,4.0,2,1,10,...,0.0,2.00,0,1.6,2.00,2.0,4064,0,0,2.2000
7,162,Delivery from Collection Point,10.0,2.2,23.98,1.0,6.6,3,28,2,...,0.0,2.20,3,1.6,2.20,2.2,4428,0,0,2.2000
10,56,Delivery from Collection Point,10.0,3.6,15.00,1.0,31.8,10,23,2,...,0.0,3.18,5,2.2,3.18,3.6,4453,0,0,1.3252
20,162,Delivery from Collection Point,10.0,2.2,23.98,1.0,6.6,3,4,2,...,0.0,2.20,0,3.6,2.20,2.2,4398,0,0,2.2000


In [93]:
sub_df = pd.merge( prb_df_encode, test_df, on='MAGIC_KEY_ENCODED')


In [95]:
sub_df.head()

,MAGIC_KEY,BOX_ID_x,QUALITY,DELIVERY_OPTION_x,MILK_x,MEAT_x,UNIT_PRICE_x,PURCHASE_DATE,BOX_COUNT_x,TOTAL_MEAT_x,...,02_LAST_PURCHASE_2019_y,PURCHASE_DURATION_y,AVG_MEAT_PURCHASE_y,PURCHASE_DAY_OF_WEEK_y,PREVIOUS_MEAT_QUANTITY_y,MEAT_MEAN_y,MEAT_MEDIAN_y,DELIVERY_ENCODED_y,QUALITY_ENCODED_y,MEAT_PURCHASE
0,2BCFE9C06A7,56,Premium,Delivery from Collection Point,10.0,3.6,15.00,2019-02-23,1.0,18.0,...,0.0,0.0,3.60,5,2.2,3.60,3.6,0,0,1.4052
1,2C2A872B5A2,132,Premium,Delivery from Collection Point,9.0,1.8,19.98,2018-10-01,1.0,4.0,...,0.0,0.0,2.00,0,1.6,2.00,2.0,0,0,2.2000
2,2C6A897671B,162,Premium,Delivery from Collection Point,10.0,2.2,23.98,2019-02-28,1.0,6.6,...,0.0,0.0,2.20,3,1.6,2.20,2.2,0,0,2.2000
3,2C6F1287F53,56,Premium,Delivery from Collection Point,10.0,3.6,15.00,2019-02-23,1.0,31.8,...,0.0,0.0,3.18,5,2.2,3.18,3.6,0,0,1.3252
4,2C658198CC9,162,Premium,Delivery from Collection Point,10.0,2.2,23.98,2019-02-04,1.0,6.6,...,0.0,0.0,2.20,0,3.6,2.20,2.2,0,0,2.2000


In [96]:
sub_df = sub_df[['MAGIC_KEY', 'MEAT_PURCHASE']]


In [97]:
sub_df = sub_df.rename(columns={'MEAT_PURCHASE': 'MEAT'})


In [102]:
sub_df.MEAT.min(), sub_df.MEAT.max()

(0.0, 20.950399999999835)

In [103]:
sub_df.to_csv('Submission2.csv', index=None)

In [79]:
#sub = sub.rename(columns={'X': 'MEAT'})


In [78]:
#columns_to_keep = ['MAGIC_KEY', 'X']
#sub = sub_df[columns_to_keep]
